In [ ]:
import torch

%pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster torch-spline-conv  --y


Found existing installation: torch-scatter 2.1.2+pt20cu118
Uninstalling torch-scatter-2.1.2+pt20cu118:
  Successfully uninstalled torch-scatter-2.1.2+pt20cu118
Found existing installation: torch-sparse 0.6.18+pt20cu118
Uninstalling torch-sparse-0.6.18+pt20cu118:
  Successfully uninstalled torch-sparse-0.6.18+pt20cu118
Found existing installation: torch_geometric 2.5.3
Uninstalling torch_geometric-2.5.3:
  Successfully uninstalled torch_geometric-2.5.3
Found existing installation: torch-cluster 1.6.3+pt20cu118
Uninstalling torch-cluster-1.6.3+pt20cu118:
  Successfully uninstalled torch-cluster-1.6.3+pt20cu118
Found existing installation: torch-spline-conv 1.2.2+pt20cu118
Uninstalling torch-spline-conv-1.2.2+pt20cu118:
  Successfully uninstalled torch-spline-conv-1.2.2+pt20cu118


In [ ]:
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

%pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
%pip install torch-geometric

Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 27.4 MB/s eta 0:00:0000:010:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 16.3 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in links: https://pytorch-geometric.com/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 11.6 MB/s eta 0:00:0000:010:01

[notice] A new release of pip

In [ ]:
%pip uninstall networkit
%pip install networkit

Found existing installation: networkit 11.0
Uninstalling networkit-11.0:
  Successfully uninstalled networkit-11.0
Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 4.1 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn
import random
from torch.utils.data import Subset
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.metrics import top_k_accuracy_score
import networkx as nx
import networkit as nk
from networkit import vizbridges
from torch_geometric.data import Data
from torch_geometric.utils import subgraph
from torch_geometric.nn import GATv2Conv, GCNConv, SAGEConv
from torch_geometric.loader import LinkNeighborLoader, LinkLoader
from torch_geometric.transforms import RandomLinkSplit


In [ ]:
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
import torch_cluster
import torch_scatter
import torch_geometric
import torch_sparse

In [ ]:
import numpy as np
import pandas as pd
import torch
from torch import nn

In [ ]:
print(torch.__version__)

2.0.1+cu118


In [ ]:
torch.manual_seed(200)
np.random.seed(200)

### Util function

In [ ]:
def load_dataset_timestamp( n_users, n_context, seq_len):
    act_list = []
    time_list = []
    user_list = []

    max_timestamp = -1.0
    min_timestamp = float('inf')

    with open('gowalla_user_activity.txt', 'r') as raw_file:
        for line in raw_file:
            t_item_list = []
            t_time_list = []
            user = int(line.split(':')[0])
            entries = line.split()[1:]
            for a_entry in entries:
                item, time_stamp = a_entry.split(':')
                t_item_list.append(int(item.strip()))
                t_time_list.append(int(time_stamp.strip()))

                if min_timestamp > int(time_stamp.strip()):
                    min_timestamp = int(time_stamp.strip())
                if max_timestamp < int(time_stamp.strip()):
                    max_timestamp = int(time_stamp.strip())

            act_list.append(t_item_list[0: seq_len])
            time_list.append(t_time_list[0: seq_len])
            user_list.append(user)

    new_time_list = []
    num_bins = 0

    times_bins = np.linspace(min_timestamp, max_timestamp + 1, num=num_bins, dtype=np.int32)
    for a_time_list in time_list:
        temp_time_list = (np.digitize(np.asarray(a_time_list), times_bins) - 1).tolist()
        new_time_list.append(temp_time_list)

    all_examples = []
    for i in range(0, len(act_list)):

        train_act_seq = act_list[i][:-2]

        train_time_seq = new_time_list[i][:-2]

        train_act_label = act_list[i][-2]
        train_time_label = new_time_list[i][-2]

        test_act_seq = act_list[i][1:-1]
        test_time_seq = new_time_list[i][1:-1]

        test_act_label = act_list[i][-1]
        test_time_label = new_time_list[i][-1]

        entry = {
            'train_act_seq': train_act_seq,
            'train_time_seq': train_time_seq,
            'train_act_label': train_act_label,
            'train_time_label': train_time_label,
            'test_act_seq': test_act_seq,
            'test_time_seq': test_time_seq,
            'test_act_label': test_act_label,
            'test_time_label': test_time_label,
            'seq_len': len(train_act_seq),
            'user': user_list[i]
        }

        all_examples.append(entry)

    return all_examples

In [ ]:
data_examples = load_dataset_timestamp(20001, 128, 30)

In [ ]:
def filter(x):
  return [user for user in users if user not in x]

### Metrics

In [ ]:
def print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100):
    print(f'hits@1: {hits1:.6f}, hits@5: {hits5:.6f}, hits@10: {hits10:.6f}, hits@20: {hits20:.6f}')
    print(f'hits@50: {hits50:.6f}, hits@100: {hits100:.6f}')
    print(f'map@1: {map1:.6f}, map@5: {map5:.6f}, map@10: {map10:.6f}, map@20: {map20:.6f}')
    print(f'map@50: {map50:.6f}, map@100: {map100:.6f}')
    print(f'ndcg@1: {ndcg1:.6f}, ndcg@5: {ndcg5:.6f}, ndcg@10: {ndcg10:.6f}, ndcg@20: {ndcg20:.6f}')
    print(f'ndcg@50: {ndcg50:.6f}, ndcg@100: {ndcg100:.6f}')

In [ ]:
def apk(actual, predicted, k=10):

    if len(predicted) > k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i, p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)


def mapk(y_prob, y, k=10):

    predicted = [np.argsort(p_)[-k:][::-1] for p_ in y_prob]
    actual = [[y_] for y_ in y]
    return np.mean([apk(a, p, k) for a, p in zip(actual, predicted)])


def hits_k(y_prob, y, k=10):
    acc = []
    for p_, y_ in zip(y_prob, y):
        top_k = p_.argsort()[-k:][::-1]
        acc += [1. if y_ in top_k else 0.]
from sklearn.metrics import ndcg_score


In [ ]:
def get_metrics_(probs, labels_batch, test_one_hot):
    hits1 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=1, labels = classes)
    hits5 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=5, labels = classes)
    hits10 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=10, labels = classes)
    hits20 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=20, labels = classes)
    hits50= top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=50, labels = classes)
    hits100 = top_k_accuracy_score(labels_batch, probs.cpu().detach().numpy(), k=100, labels = classes)

    map1 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=1)
    map5 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=5)
    map10 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=10)
    map20 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=20)
    map50 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=50)
    map100 = mapk(y_prob=probs.cpu().detach().numpy(), y = labels_batch, k=100)

    ndcg1 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=1)
    ndcg5 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=5)
    ndcg10 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=10)
    ndcg20 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=20)
    ndcg50 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=50)
    ndcg100 = ndcg_score(test_one_hot, probs.cpu().detach().numpy(), k=100)
    return hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100

### User representations dataset

In [ ]:
class UserDataset():
    def __init__(self, user_representations):
        self.user_representations = user_representations

    def __len__(self):
        return len(self.user_representations)

    def __getitem__(self, idx):
        user_id = idx
        if idx not in self.user_representations.keys():
          return user_id, self.user_representations[1][0], 0
        return user_id, self.user_representations[idx][0], self.user_representations[idx][1].item()

In [ ]:
num_classes = 3883
classes = np.arange(0, num_classes)

In [ ]:
item_emb  = nn.init.xavier_uniform_(torch.empty(num_classes, 128))

### RNN + Graph Link prediction

In [ ]:
num_classes = 3883
classes = np.arange(0, num_classes)

In [ ]:
users = load_dataset_timestamp(20001, 128, 30)

In [ ]:
friends = pd.read_csv('gowalla_edges.csv')

Graph construction

In [ ]:
Gcl = nx.Graph()
for i in range(len(users)):
  if users[i]['user'] not in Gcl.nodes():
      Gcl.add_node(users[i]['user'], weight=10, color='seagreen')
for i in range(len(friends)):
  Gcl.add_edge(friends.loc[i, '1st friend'], friends.loc[i, '2nd friend'], color = 'blue')


nodes = list(Gcl.nodes())
node_colors = [Gcl.nodes[node]['color'] for node in Gcl.nodes]
node_weights = [Gcl.nodes[node]['weight'] for node in Gcl.nodes]
edge_colors = [Gcl.edges[edge]['color'] for edge in Gcl.edges]

In [ ]:
graphid2pid = {}
pid2graphid = {}

s = 0
for node_id in Gcl.nodes():
    pid2graphid[node_id] = s
    graphid2pid[s] = node_id
    s += 1

graph_edges = []
for (u, v) in Gcl.edges():
    graph_edges.append((pid2graphid[u], pid2graphid[v]))

In [ ]:
class UserInfoDataset():
  def __init__(self, data, max_len):
    self.data = data
    self.max_len = max_len

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    user = self.data[idx]
    seq_len = user['seq_len']

    tr_act_seq = np.zeros((self.max_len,)).astype('int32')
    tr_act_seq[:seq_len] = np.array(user['train_act_seq'])
    tr_act_seq = np.transpose(tr_act_seq)

    tr_time_seq = np.zeros((self.max_len,)).astype('int32')
    tr_time_seq[:seq_len] = user['train_time_seq']
    tr_time_seq = np.transpose(tr_time_seq)

    t_act_seq = np.zeros((self.max_len, )).astype('int32')
    t_act_seq[:seq_len] = user['test_act_seq']
    t_act_seq = np.transpose(t_act_seq)

    t_time_seq = np.zeros((self.max_len, )).astype('int32')
    t_time_seq[:seq_len] = user['test_time_seq']
    t_time_seq = np.transpose(t_time_seq)


    return user['user'], tr_act_seq, \
    tr_time_seq, user['train_act_label'], \
    user['train_time_label'], t_act_seq, \
    t_time_seq, user['test_act_label'], \
    user['test_time_label'], user['seq_len']

In [ ]:
from torch.utils.data import Subset
from torch.utils.data import DataLoader

user_dataset = UserInfoDataset(users, 30)

n = len(user_dataset)

indices = np.arange(n)
indices = np.random.permutation(indices)

train_indices = indices [:int(0.8*n)]
test_indices = indices[int(0.8*n):]

user_train_dataset = Subset(user_dataset, train_indices)
user_test_dataset = Subset(user_dataset, test_indices)

user_dataloader = DataLoader(user_dataset, batch_size = 20001, shuffle = True)
user_train_dataloader = DataLoader(user_train_dataset, batch_size=64, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=64, shuffle=True)

item_emb  = nn.init.xavier_uniform_(torch.empty(num_classes, 128))

In [ ]:
user_representations = {}
for users, train_input, train_time, train_label, train_time_label, test_input, test_time, test_label, test_time_label, seq_len in user_dataloader:
      comb_input = np.concatenate([np.expand_dims(train_input, axis=-1),
                                                np.expand_dims(train_time, axis=-1)], axis=2)
      model_input = comb_input
      model_output = train_label
      test_comb_input = np.concatenate([np.expand_dims(test_input, axis=-1),
                                                np.expand_dims(test_time, axis=-1)], axis=2)
      test_model_input = test_comb_input
      rnn_input_emb = item_emb[model_input[:, :, 0]]
      test_rnn_input_emb = item_emb[test_model_input[:, :, 0]]
      for i in range(len(users)):
        user_representations[users[i].item()] = (users[i].item(), rnn_input_emb[i], train_label[i].item(), seq_len[i].item(), test_rnn_input_emb[i], test_label[i].item())

In [ ]:
user_ids = []
features = []
train_labels = []
seq_lens = []
test_features = []
test_labels = []
for graphid in Gcl.nodes():
    pid = graphid2pid[graphid]
    if graphid in user_representations.keys():
      user_ids.append(user_representations[graphid][0])
      features.append(user_representations[graphid][1].detach().numpy())
      train_labels.append(user_representations[graphid][2])
      test_features.append(user_representations[graphid][4].detach().numpy())
      seq_lens.append(user_representations[graphid][3])
      test_labels.append(user_representations[graphid][5])
user_ids = np.array(user_ids)
features = np.array(features)
train_labels = np.array(train_labels)
test_features = np.array(test_features)
seq_lens = np.array(seq_lens)
test_labels = np.array(test_labels)

In [ ]:
dataset = Data(x=torch.tensor(features, dtype=torch.float), edge_index=torch.tensor(np.array(graph_edges).T, dtype=torch.int64))

In [ ]:
dataset.user_ids = torch.tensor(user_ids, dtype=torch.int64)
dataset.train_labels = torch.tensor(train_labels, dtype=torch.int64)
dataset.seq_lens = torch.tensor(seq_lens, dtype=torch.int64)
dataset.test_labels = torch.tensor(test_labels, dtype=torch.int64)
dataset.test_features = torch.tensor(test_features, dtype=torch.float)

In [ ]:
link_splitter = RandomLinkSplit(
     num_val=0.0,
     num_test=0.0,
     add_negative_train_samples=False,
     disjoint_train_ratio=0.8)

In [ ]:
train_g, val_g, test_g = link_splitter(dataset)

In [ ]:
test_dataset = Data(x=torch.tensor(test_features, dtype=torch.float), edge_index=torch.tensor(np.array(graph_edges).T, dtype=torch.int64))
test_dataset.user_info = users_info

In [ ]:
test_loader = LinkNeighborLoader(
     test_dataset,
     num_neighbors=[-1, 10, 5],
     batch_size=128,
     neg_sampling_ratio=0.5
 )

In [ ]:
def negative_sampling(edge_index, num_nodes):
    mask_1 = torch.rand(edge_index.size(1)) < 0.5
    mask_2 = ~mask_1

    neg_edge_index = edge_index.clone()
    neg_edge_index[0, mask_1] = torch.randint(num_nodes, (mask_1.sum(),))
    neg_edge_index[1, mask_2] = torch.randint(num_nodes, (mask_2.sum(),))

    return neg_edge_index

In [ ]:
device = 'cuda'

RNN + SAGE Conv, берем выходы RNN

In [ ]:
class GraphLinkPredictionModel(nn.Module):

  def __init__(self):
    super(GraphLinkPredictionModel, self).__init__()

    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.dropout = nn.Dropout(p=0.3)
    self.relu = nn.Tanh()
    self.norm = nn.BatchNorm1d(128)
    self.head_1 = nn.Sequential(nn.Linear(256, 1))
    self.sigmoid= nn.Sigmoid()
    self.graphconv = SAGEConv(128, 128)
    self.graphconv2 = SAGEConv(128, 128)
    self.prelu1 = nn.PReLU(128)
    self.head_2 = nn.Linear(128, num_classes)

  def forward(self, x, seq_len, edge_index):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]

    hx = hx.to(device)
    hx = self.norm(hx)
    out_hx = hx
    hx = self.graphconv(hx, edge_index)
    hx = self.prelu1(hx)
    hx = self.graphconv2(hx, edge_index)
    hx = self.prelu1(hx)
    hx = self.graphconv2(hx, edge_index)
    x_src, x_dst = hx[edge_index[0]], hx[edge_index[1]]
    x = torch.cat((x_src, x_dst), dim = 1)
    x = self.head_1(x)
    probs = 0
    return x, probs, out_hx

RNN + SAGE Conv, берем выходы SAGE Conv

In [ ]:
class GraphLinkPredictionModel(nn.Module):

  def __init__(self):
    super(GraphLinkPredictionModel, self).__init__()

    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.dropout = nn.Dropout(p=0.3)
    self.relu = nn.Tanh()
    self.norm = nn.BatchNorm1d(128)
    self.head_1 = nn.Sequential(nn.Linear(256, 1))
    self.sigmoid= nn.Sigmoid()
    self.graphconv = SAGEConv(128, 128)
    self.graphconv2 = SAGEConv(128, 128)
    self.prelu1 = nn.PReLU(128)
    self.head_2 = nn.Linear(128, num_classes)

  def forward(self, x, seq_len, edge_index):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]
    hx = hx.to(device)
    hx = self.norm(hx)
    out_hx = hx
    hx = self.graphconv(hx, edge_index)
    hx = self.prelu1(hx)
    hx = self.graphconv2(hx, edge_index)
    hx = self.prelu1(hx)
    hx = self.graphconv2(hx, edge_index)
    x_src, x_dst = hx[edge_index[0]], hx[edge_index[1]]
    x = torch.cat((x_src, x_dst), dim = 1)
    x = self.head_1(x)
    probs = 0
    return x, probs, hx

Просто RNN

In [ ]:
class GraphLinkPredictionModel(nn.Module):

  def __init__(self):
    super(GraphLinkPredictionModel, self).__init__()

    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.dropout = nn.Dropout(p=0.3)
    self.relu = nn.Tanh()
    self.norm = nn.BatchNorm1d(128)
    self.head_1 = nn.Sequential(nn.Linear(256, 1))
    self.sigmoid= nn.Sigmoid()
    self.graphconv = SAGEConv(128, 128)
    self.graphconv2 = SAGEConv(128, 128)
    self.prelu1 = nn.PReLU(128)
    self.head_2 = nn.Linear(128, num_classes)

  def forward(self, x, seq_len, edge_index):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]
    hx = hx.to(device)
    hx = self.norm(hx)

    x_src, x_dst = hx[edge_index[0]], hx[edge_index[1]]
    x = torch.cat((x_src, x_dst), dim = 1)
    x = self.head_1(x)
    probs = 0
    return x, probs, hx

In [ ]:
%pip install torchmetrics
from torchmetrics.classification import BinaryAccuracy

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 24.0
[notice] To update, run: python3 -m pip install --upgrade pip


In [ ]:
torch.cuda.reset_peak_memory_stats(device)
print(torch.cuda.max_memory_allocated(0) / (1024 ** 3))

0.0


In [ ]:
torch.cuda.reset_peak_memory_stats(device)
next_loss_fn = nn.CrossEntropyLoss()
model = GraphLinkPredictionModel()
link_loss_fn = nn.BCEWithLogitsLoss()
accuracy = BinaryAccuracy().to(device)
model.to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)

train_user_representations = {}
test_user_representations = {}
num_epochs = 10
accuracies = []
test_accuracies = []
for i in range(num_epochs):
  losses = []
  hits_1_scores = []
  batch_num = 0
  for batch in train_loader:
      batch_num += 1
      optimizer.zero_grad()
      batch = batch.to(device)

      link_probs, next_probs, hx = model(batch.x, batch.seq_lens, batch.edge_label_index)
      test_link_probs, test_next_probs, test_hx = model(batch.test_features, batch.seq_lens, batch.edge_label_index)
      link_probs = link_probs.squeeze()
      test_link_probs = test_link_probs.squeeze()

      gt = batch.edge_label
      link_loss = link_loss_fn(link_probs, gt)
      accuracies.append(accuracy(link_probs, gt))
      test_accuracies.append(accuracy(test_link_probs, gt))

      loss = 1 * link_loss
      losses.append(loss)
      loss.backward()
      optimizer.step()


      if i == num_epochs - 1:
        index = 0
        for u in batch.user_ids:
          user = u.item()
          if user not in train_user_representations.keys():
            train_user_representations[user] = (hx[index].detach(), batch.train_labels[index])
          if user not in test_user_representations.keys():
            test_user_representations[user] = (test_hx[index].detach(), batch.test_labels[index])
          index += 1

  mean_loss = torch.Tensor(losses).mean(axis=0)
  mean_acc = torch.Tensor(accuracies).mean(axis=0).item()
  mean_test_acc = torch.Tensor(test_accuracies).mean(axis=0).item()
  print(f'Epoch: {i} Loss: {mean_loss.item()} Train Accuracy: {mean_acc} Test Accuracy: {mean_test_acc}')
  print(torch.cuda.max_memory_allocated(0) / (1024 ** 3))

Epoch: 0 Loss: 0.5328154563903809 Train Accuracy: 0.7150883078575134 Test Accuracy: 0.6756420135498047
0.08614587783813477
Epoch: 1 Loss: 0.467550128698349 Train Accuracy: 0.739663302898407 Test Accuracy: 0.6740946173667908
0.09111881256103516
Epoch: 2 Loss: 0.4365590810775757 Train Accuracy: 0.755257248878479 Test Accuracy: 0.6690285801887512
0.09484338760375977
Epoch: 3 Loss: 0.4140571057796478 Train Accuracy: 0.7670415639877319 Test Accuracy: 0.6642501354217529
0.09961557388305664
Epoch: 4 Loss: 0.3987140953540802 Train Accuracy: 0.7761774659156799 Test Accuracy: 0.6588885188102722
0.10285043716430664
Epoch: 5 Loss: 0.38458681106567383 Train Accuracy: 0.7837252616882324 Test Accuracy: 0.6538312435150146
0.10593891143798828
Epoch: 6 Loss: 0.3747740685939789 Train Accuracy: 0.7901202440261841 Test Accuracy: 0.6488096714019775
0.11064863204956055
Epoch: 7 Loss: 0.36430999636650085 Train Accuracy: 0.7956680059432983 Test Accuracy: 0.644772469997406
0.11440610885620117
Epoch: 8 Loss: 0.3

In [ ]:
torch.cuda.max_memory_allocated(0) / (1024 ** 3)

0.39248037338256836

### Linear Layer Target training on pre-trained embeddings

In [ ]:
user_train_dataset = UserDataset(train_user_representations)
user_test_dataset = UserDataset(test_user_representations)

user_train_dataloader = DataLoader(user_train_dataset, batch_size=65, shuffle=True)
user_test_dataloader = DataLoader(user_test_dataset, batch_size=65, shuffle=True)

In [ ]:
class TargetModel(nn.Module):
  def __init__(self):
    super(TargetModel, self).__init__()
    self.fc1 = nn.Linear(128, num_classes)

  def forward(self, x):
    x = self.fc1(x)

    return x

In [ ]:
def test(model):
  metrics_val = []
  model.eval()

  for users, vectors, labels in user_test_dataloader:

      test_probs = model(vectors.cpu())

      test_pred = torch.argmax(test_probs, axis = 1)

      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), labels] = 1

      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100,\
      ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = get_metrics_(test_probs, labels, test_one_hot)
      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])

  mean = torch.Tensor(metrics_val).mean(axis=0)
  test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
  test_ndcg1, test_ndcg5,test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean

  return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100,\
  test_ndcg1, test_ndcg5,test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100

In [ ]:
def middle_test(f):
  hits_1 = []
  for users, vectors, labels in user_test_dataloader:

    test_probs = model(vectors.cpu())

    test_pred = torch.argmax(test_probs, axis = 1)

    test_one_hot = torch.zeros(len(test_probs), num_classes)
    test_one_hot[torch.arange(len(test_one_hot)), labels] = 1
    hits_1.append(top_k_accuracy_score(labels, test_probs.cpu().detach().numpy(), k=1, labels = classes))
  print(f'Hits@1: {torch.mean(torch.Tensor(hits_1))}')
  f.write(f'Hits@1: {torch.mean(torch.Tensor(hits_1))}')


In [ ]:
torch.cuda.reset_peak_memory_stats(device)
torch.cuda.max_memory_allocated(0) / (1024 ** 3)

0.33942604064941406

In [ ]:
f = open('linear_layer_target_training_3883.txt', 'w+')
torch.cuda.reset_peak_memory_stats(device)
loss_fn = nn.CrossEntropyLoss()
model  = TargetModel()

model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)
losses = []
hits_1 = []

for i in range(200):
  for users, vectors, labels in user_train_dataloader:

      optimizer.zero_grad()

      probs = model(vectors.cpu())
      pred = torch.argmax(probs, axis = 1)
      one_hot = torch.zeros(len(probs), num_classes)
      one_hot[torch.arange(len(one_hot)), labels] = 1
      loss = loss_fn(probs, one_hot)
      losses.append(loss)
      loss.backward()
      optimizer.step()

  print(f'Epoch: {i} Loss: {torch.mean(torch.Tensor(losses))}')
  f.write(f'Epoch: {i} Loss: {torch.mean(torch.Tensor(losses))}')
  print(torch.cuda.max_memory_allocated(0) / (1024 ** 3))
  middle_test(f)
hits1, hits5, hits10, hits20, hits50, hits100, map1,map5, map10, map20, map50, map100, \
ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = test(model)
print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)
write_to_file(f, hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)
f.close()

Epoch: 0 Loss: 8.157064437866211
0.33942604064941406
Epoch: 1 Loss: 7.882716655731201
0.33942604064941406
Epoch: 2 Loss: 7.633568286895752
0.33942604064941406
Epoch: 3 Loss: 7.400608539581299
0.33942604064941406
Epoch: 4 Loss: 7.180976867675781
0.33942604064941406
Epoch: 5 Loss: 6.97373628616333
0.33942604064941406
Epoch: 6 Loss: 6.778742790222168
0.33942604064941406
Epoch: 7 Loss: 6.596052646636963
0.33942604064941406
Epoch: 8 Loss: 6.425450801849365
0.33942604064941406
Epoch: 9 Loss: 6.266213417053223
0.33942604064941406
Epoch: 10 Loss: 6.117584228515625
0.33942604064941406
Epoch: 11 Loss: 5.978617191314697
0.33942604064941406
Epoch: 12 Loss: 5.848659515380859
0.33942604064941406
Epoch: 13 Loss: 5.726806640625
0.33942604064941406
Epoch: 14 Loss: 5.6122918128967285
0.33942604064941406
Epoch: 15 Loss: 5.5044684410095215
0.33942604064941406
Epoch: 16 Loss: 5.402859210968018
0.33942604064941406
Epoch: 17 Loss: 5.306805610656738
0.33942604064941406
Epoch: 18 Loss: 5.2158660888671875
0.339

In [ ]:
torch.cuda.max_memory_allocated(0) / (1024 ** 3)

0.33942604064941406

### RNN Target training with Concatenation

In [ ]:
users_list = load_dataset_timestamp(20001, 128, 30)

In [ ]:
import random
class UserInfoDataset():
  def __init__(self, data, max_len, train_user_representations, test_user_representations, coeff):
    self.data = data
    self.max_len = max_len
    self.train_user_representations = train_user_representations
    self.test_user_representations = test_user_representations
    self.coeff = coeff

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):

    user = self.data[idx]
    seq_len = user['seq_len']
    seq_len = len(user['train_act_seq'])
    indexes = np.arange(0, seq_len)
    if seq_len - self.coeff > 0:
      seq_len = seq_len - self.coeff
    random_indexes = random.sample(list(indexes), seq_len)
    user['train_act_seq'] = np.array(user['train_act_seq'])
    user['train_act_seq'] = user['train_act_seq'][sorted(random_indexes)]
    user['test_act_seq'] = np.array(user['test_act_seq'])
    user['test_act_seq'] = user['test_act_seq'][sorted(random_indexes)]
    user['train_time_seq'] = np.array(user['train_time_seq'])
    user['train_time_seq'] = user['train_time_seq'][sorted(random_indexes)]
    user['test_time_seq'] = np.array(user['test_time_seq'])
    user['test_time_seq'] = user['test_time_seq'][sorted(random_indexes)]

    tr_act_seq = np.zeros((self.max_len,)).astype('int32')
    tr_act_seq[:seq_len] = np.array(user['train_act_seq'])
    tr_act_seq = np.transpose(tr_act_seq)

    tr_time_seq = np.zeros((self.max_len,)).astype('int32')
    tr_time_seq[:seq_len] = user['train_time_seq']
    tr_time_seq = np.transpose(tr_time_seq)

    t_act_seq = np.zeros((self.max_len, )).astype('int32')
    t_act_seq[:seq_len] = user['test_act_seq']
    t_act_seq = np.transpose(t_act_seq)

    t_time_seq = np.zeros((self.max_len, )).astype('int32')
    t_time_seq[:seq_len] = user['test_time_seq']
    t_time_seq = np.transpose(t_time_seq)

    if idx not in self.train_user_representations.keys():
      self.train_user_representations[idx] = (torch.zeros(size=(128,)), torch.tensor(0))
      self.test_user_representations[idx] = (torch.zeros(size=(128,)), torch.tensor(0))


    return user['user'], tr_act_seq, \
    tr_time_seq, user['train_act_label'], \
    user['train_time_label'], t_act_seq, \
    t_time_seq, user['test_act_label'], \
    user['test_time_label'], seq_len, self.train_user_representations[idx][0].cpu(), self.test_user_representations[idx][0].cpu()

Corruption Коэффициент для экспериментов на проверку устойчивости

In [ ]:
coeff = 0

In [ ]:
user_dataset = UserInfoDataset(users_list, 30, train_user_representations, test_user_representations, coeff)
user_dataloader = DataLoader(user_dataset, batch_size = 64, shuffle=True)

Конкатенация с предобученным эмбеддингом

In [ ]:
class RNNTargetModel(nn.Module):
  def __init__(self, num_classes):
    super(RNNTargetModel, self).__init__()
    self.fc1 = nn.Linear(256, num_classes)
    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.norm = nn.BatchNorm1d(128)
    self.reduce_dim = nn.Linear(128, 128)

  def forward(self, x, repr, seq_len):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]
    hx = hx.to(device)
    hx = self.norm(hx)
    hx = torch.cat([hx, repr], dim= 1)
    x = self.fc1(hx)
    return x

Конкатенация со случайно сгенерированным эмбеддингом

In [ ]:
class RNNTargetModel(nn.Module):
  def __init__(self, num_classes):
    super(RNNTargetModel, self).__init__()
    self.fc1 = nn.Linear(256, num_classes)
    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.norm = nn.BatchNorm1d(128)
    self.reduce_dim = nn.Linear(128, 128)

  def forward(self, x, repr, seq_len):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]
    hx = hx.to(device)
    hx = self.norm(hx)
    repr = torch.rand((len(x), 128))
    hx = torch.cat([hx, repr], dim= 1)
    x = self.fc1(hx)
    return x

In [ ]:
def rnn_target_test(model):
  metrics_val = []
  model.eval()
  index = 0

  for user, train_input, train_time, train_label, train_time_label, test_input, test_time, test_label, test_time_label, seq_len, train_repr, test_repr in user_dataloader:

      test_comb_input = np.concatenate([np.expand_dims(test_input, axis=-1),
                                                np.expand_dims(test_time, axis=-1)], axis=2)
      model_input = test_comb_input
      model_output = test_label
      test_rnn_input_emb = item_emb[model_input[:, :, 0]]
      test_probs = model(test_rnn_input_emb.cuda(), test_repr, seq_len)

      test_pred = torch.argmax(test_probs, axis = 1)

      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), test_label] = 1
      loss = loss_fn(test_probs, test_one_hot.cuda())

      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, \
      ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = get_metrics_(test_probs, test_label, test_one_hot)

      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])


  mean = torch.Tensor(metrics_val).mean(axis=0)
  test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, \
  test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean
  return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100,\
  test_ndcg1, test_ndcg5, test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100


In [ ]:
def write_to_file(f, hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100):
    f.write(f'hits@1: {hits1:.6f}, hits@5: {hits5:.6f}, hits@10: {hits10:.6f}, hits@20: {hits20:.6f}\n')
    f.write(f'hits@50: {hits50:.6f}, hits@100: {hits100:.6f}\n')
    f.write(f'map@1: {map1:.6f}, map@5: {map5:.6f}, map@10: {map10:.6f}, map@20: {map20:.6f}\n')
    f.write(f'map@50: {map50:.6f}, map@100: {map100:.6f}\n')
    f.write(f'ndcg@1: {ndcg1:.6f}, ndcg@5: {ndcg5:.6f}, ndcg@10: {ndcg10:.6f}, ndcg@20: {ndcg20:.6f}\n')
    f.write(f'ndcg@50: {ndcg50:.6f}, ndcg@100: {ndcg100:.6f}\n')

In [ ]:
torch.cuda.reset_peak_memory_stats(device)
print(torch.cuda.max_memory_allocated(0) / (1024 ** 3))

0.3348393440246582


In [ ]:
f = open(f'gowalla_rnn_stability_{coeff}.txt', 'w+')
# for iteration in range(3):
# torch.manual_seed(10 * iteration)
torch.cuda.reset_peak_memory_stats(device)
model = RNNTargetModel(num_classes)
model.to(device)
loss_fn = nn.CrossEntropyLoss()
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)


for i in range(20):
    losses = []
    val_losses = []
    hits_1_scores = []

    for user, train_input, train_time, train_label, train_time_label, test_input, test_time, test_label, \
    test_time_label, seq_len, train_repr, test_repr in user_dataloader:
        optimizer.zero_grad()
        comb_input = np.concatenate([np.expand_dims(train_input, axis=-1),
                                                  np.expand_dims(train_time, axis=-1)], axis=2)
        model_input = comb_input
        model_output = train_label
        rnn_input_emb = item_emb[model_input[:, :, 0]]
        test_comb_input = np.concatenate([np.expand_dims(test_input, axis=-1),
                                                  np.expand_dims(test_time, axis=-1)], axis=2)
        test_model_input = test_comb_input
        test_rnn_input_emb = item_emb[test_model_input[:, :, 0]]
        probs = model(rnn_input_emb.cuda(), train_repr.cuda(), seq_len)
        test_probs = model(test_rnn_input_emb.cuda(), test_repr.cuda(), seq_len)
        pred = torch.argmax(probs, axis = 1)

        one_hot = torch.zeros(len(probs), num_classes)
        one_hot[torch.arange(len(one_hot)), model_output] = 1

        test_one_hot = torch.zeros(len(test_probs), num_classes)
        test_one_hot[torch.arange(len(test_one_hot)), test_label] = 1

        loss = loss_fn(probs, one_hot.cuda())
        val_loss = loss_fn(test_probs, test_one_hot.cuda())
        losses.append(loss)
        val_losses.append(val_loss)
        hits_1_scores.append(top_k_accuracy_score(test_label, test_probs.cpu().detach().numpy(), k=1, labels = classes))
        loss.backward()
        optimizer.step()
        losses.append(loss)


    mean_loss = torch.Tensor(losses).mean(axis=0)
    mean_val_loss = torch.Tensor(val_losses).mean(axis = 0).item()
    mean_hits = torch.Tensor(hits_1_scores).mean(axis=0).item()
    print(f'Epoch: {i} Loss: {mean_loss.item()}, Val loss: {mean_val_loss} Hits@1: {mean_hits}')
    print(torch.cuda.max_memory_allocated(0) / (1024 ** 3))
    hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = rnn_target_test(model)
    # f.write(f'Try: {iteration}, Seed: {10*iteration}\n')
    # write_to_file(f, hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)
    print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)
# f.close()

Epoch: 0 Loss: 7.748237133026123, Val loss: 7.765230178833008 Hits@1: 0.08204448223114014
1.8501653671264648
Epoch: 1 Loss: 6.409521102905273, Val loss: 6.5093865394592285 Hits@1: 0.2646901309490204
1.8501653671264648
Epoch: 2 Loss: 5.349963188171387, Val loss: 5.513329029083252 Hits@1: 0.3352590501308441
1.8501653671264648
Epoch: 3 Loss: 4.551754474639893, Val loss: 4.765737533569336 Hits@1: 0.39165517687797546
1.8501653671264648
Epoch: 4 Loss: 3.9575483798980713, Val loss: 4.2119364738464355 Hits@1: 0.4385967254638672
1.8501653671264648
Epoch: 5 Loss: 3.5184848308563232, Val loss: 3.8163297176361084 Hits@1: 0.4706045985221863
1.8501653671264648
Epoch: 6 Loss: 3.194004535675049, Val loss: 3.5367190837860107 Hits@1: 0.4933681786060333
1.8501653671264648
Epoch: 7 Loss: 2.9439852237701416, Val loss: 3.333458185195923 Hits@1: 0.5085937976837158
1.8501653671264648
Epoch: 8 Loss: 2.7471814155578613, Val loss: 3.178215265274048 Hits@1: 0.520266056060791
1.8501653671264648
Epoch: 9 Loss: 2.58

In [ ]:
torch.cuda.mem_get_info()[0] / 1024 ** 3

29.3050537109375

In [ ]:
torch.cuda.max_memory_allocated(0) / (1024 ** 3)

1.8501653671264648

### Joint learning

In [ ]:
class GraphLinkPredictionModel(nn.Module):

  def __init__(self):
    super(GraphLinkPredictionModel, self).__init__()

    self.rnn = nn.RNN(128, 128, batch_first = True)
    self.dropout = nn.Dropout(p=0.3)
    self.relu = nn.Tanh()
    self.norm = nn.BatchNorm1d(128)
    self.head_1 = nn.Sequential(nn.Linear(256, 1))
    self.head_2 = nn.Linear(128, num_classes)
    self.sigmoid= nn.Sigmoid()
    self.graphconv = SAGEConv(128, 128)
    self.graphconv2 = SAGEConv(128, 128)
    self.prelu1 = nn.PReLU(128)
    self.head_2 = nn.Linear(128, num_classes)

  def forward(self, x, seq_len, edge_index):
    x, h = self.rnn(x)
    hx = torch.zeros(x.shape[0], x.shape[2])
    for i in range(hx.shape[0]):
      hx[i] = x[i][seq_len[i] - 1]
    hx = hx.to(device)
    hx = self.norm(hx)
    out_hx = hx
    hx = self.graphconv(hx, edge_index)
    hx = self.prelu1(hx)
    hx = self.graphconv2(hx, edge_index)
    hx = self.prelu1(hx)
    hx = self.graphconv2(hx, edge_index)
    x_src, x_dst = hx[edge_index[0]], hx[edge_index[1]]
    x = torch.cat((x_src, x_dst), dim = 1)
    x = self.head_1(x)
    probs = self.head_2(hx)
    return x, probs, hx

In [ ]:
def joint_learning_test(model):
  metrics_val = []
  model.eval()
  batch_num = 0
  for batch in train_loader:
      batch = batch.to(device)
      batch_num += 1
      _, test_probs, test_hx = model(batch.test_features, batch.seq_lens, batch.edge_label_index)
      test_pred = torch.argmax(test_probs, axis = 1)
      labels = batch.test_labels
      test_one_hot = torch.zeros(len(test_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), labels] = 1

      hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100,\
      ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = get_metrics_(test_probs, labels.cpu(), test_one_hot.cpu())
      metrics_val.append([hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100])
      if batch_num > 700:
            break

  mean = torch.Tensor(metrics_val).mean(axis=0)
  test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100, \
  test_ndcg1, test_ndcg5,test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100 = mean

  return test_hits1, test_hits5, test_hits10, test_hits20, test_hits50, test_hits100, test_map1, test_map5, test_map10, test_map20, test_map50, test_map100,\
  test_ndcg1, test_ndcg5,test_ndcg10, test_ndcg20, test_ndcg50, test_ndcg100

In [ ]:
torch.cuda.reset_peak_memory_stats(device)
next_loss_fn = nn.CrossEntropyLoss()
model = GraphLinkPredictionModel()
link_loss_fn = nn.BCEWithLogitsLoss()
accuracy = BinaryAccuracy().to(device)
model.to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)
lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=optimizer, gamma=0.97, last_epoch=-1)

train_user_representations = {}
test_user_representations = {}
num_epochs = 10
accuracies = []
test_accuracies = []
for i in range(num_epochs):
  losses = []
  hits_1_scores = []
  batch_num = 0
  for batch in train_loader:
      batch_num += 1
      optimizer.zero_grad()
      batch = batch.to(device)

      link_probs, next_probs, hx = model(batch.x, batch.seq_lens, batch.edge_label_index)
      test_link_probs, test_next_probs, test_hx = model(batch.test_features, batch.seq_lens, batch.edge_label_index)
      link_probs = link_probs.squeeze()
      test_link_probs = test_link_probs.squeeze()

      one_hot = torch.zeros(len(next_probs), num_classes)
      one_hot[torch.arange(len(one_hot)), batch.train_labels] = 1
      test_one_hot = torch.zeros(len(test_next_probs), num_classes)
      test_one_hot[torch.arange(len(test_one_hot)), batch.test_labels] = 1
      hits_1_scores.append(top_k_accuracy_score(batch.test_labels, test_next_probs.cpu().detach().numpy(), k=1, labels = classes))

      gt = batch.edge_label
      link_loss = link_loss_fn(link_probs, gt)
      accuracies.append(accuracy(link_probs, gt))
      test_accuracies.append(accuracy(test_link_probs, gt))
      next_loss = next_loss_fn(next_probs, one_hot)
      loss = 1 * link_loss
      loss = next_loss
      losses.append(loss)
      loss.backward()
      optimizer.step()

      if i == num_epochs - 1:
        index = 0
        for u in batch.user_ids:
          user = u.item()
          if user not in train_user_representations.keys():
            train_user_representations[user] = (hx[index].detach(), batch.train_labels[index])
          if user not in test_user_representations.keys():
            test_user_representations[user] = (test_hx[index].detach(), batch.test_labels[index])
          index += 1

  mean_loss = torch.Tensor(losses).mean(axis=0)
  mean_hits = torch.Tensor(hits_1_scores).mean(axis=0).item()
  mean_acc = torch.Tensor(accuracies).mean(axis=0).item()
  mean_test_acc = torch.Tensor(test_accuracies).mean(axis=0).item()
  print(f'Epoch: {i} Loss: {mean_loss.item()}, Test hits@1: {mean_hits}')
  print(f'Epoch: {i} Loss: {mean_loss.item()} Train Accuracy: {mean_acc} Test Accuracy: {mean_test_acc}')
  print(torch.cuda.max_memory_allocated(0) / (1024 ** 3))
hits1, hits5, hits10, hits20, hits50, hits100, map1,map5, map10, map20, map50, map100, \
ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = joint_learning_test(model)
print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)

Epoch: 0 Loss: 0.5328154563903809 Train Accuracy: 0.7150883078575134 Test Accuracy: 0.6756420135498047
0.08614587783813477
Epoch: 1 Loss: 0.467550128698349 Train Accuracy: 0.739663302898407 Test Accuracy: 0.6740946173667908
0.09111881256103516
Epoch: 2 Loss: 0.4365590810775757 Train Accuracy: 0.755257248878479 Test Accuracy: 0.6690285801887512
0.09484338760375977
Epoch: 3 Loss: 0.4140571057796478 Train Accuracy: 0.7670415639877319 Test Accuracy: 0.6642501354217529
0.09961557388305664
Epoch: 4 Loss: 0.3987140953540802 Train Accuracy: 0.7761774659156799 Test Accuracy: 0.6588885188102722
0.10285043716430664
Epoch: 5 Loss: 0.38458681106567383 Train Accuracy: 0.7837252616882324 Test Accuracy: 0.6538312435150146
0.10593891143798828
Epoch: 6 Loss: 0.3747740685939789 Train Accuracy: 0.7901202440261841 Test Accuracy: 0.6488096714019775
0.11064863204956055
Epoch: 7 Loss: 0.36430999636650085 Train Accuracy: 0.7956680059432983 Test Accuracy: 0.644772469997406
0.11440610885620117
Epoch: 8 Loss: 0.3

In [ ]:
hits1, hits5, hits10, hits20, hits50, hits100, map1,map5, map10, map20, map50, map100, \
ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100 = joint_learning_test(model)
print_metrics(hits1, hits5, hits10, hits20, hits50, hits100, map1, map5, map10, map20, map50, map100, ndcg1, ndcg5, ndcg10, ndcg20, ndcg50, ndcg100)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
